In [9]:
! pip install tensorboardX

In [1]:
# import argparse
from itertools import count

import os, sys, random
import numpy as np
import pandas as pd

import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from tensorboardX import SummaryWriter
import sys
import os
from continuous_cartpole import ContinuousCartPoleEnv
env_path = os.path.join(os.path.abspath(os.getcwd()), '..\\Environments\\ContinuousCartPole')
sys.path.append(env_path)

# env = gym.make('CartPole-v1').unwrapped
env = ContinuousCartPoleEnv()
'''
Implementation of Deep Deterministic Policy Gradients (DDPG) with pytorch 
original paper: https://arxiv.org/abs/1509.02971
Not the author's implementation !
'''

mode ='train'
# OpenAI gym environment name, # ['BipedalWalker-v2', 'Pendulum-v0'] or any continuous environment
# Note that DDPG is feasible about hyper-parameters.
# You should fine-tuning if you change to another environment.
env_name = "ContinuousCartPoleEnv"
tau = 0.005 # target smoothing coefficient
target_update_interval = 1
test_iteration = 10

learning_rate =1e-4
gamma = 0.99 # discounted factor
capacity = 1000000 # replay buffer size
batch_size = 100 # mini batch size
seed = False
random_seed = 9527
# optional parameters

sample_frequency = 2000
render = False # show UI or not
log_interval =50 #
load =False # load model
render_interval = 100 # after render_interval, the env.render() will work
exploration_noise = 0.1
noise_clip = 0.5
policy_noise=0.2
policy_freq = 2
max_episode = 1000  # num of episodes
print_log = 5
update_iteration = 200

/home/stoch-lab/anaconda3/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# script_name = os.path.basename('/usr/local/lib/python3.7/dist-packages/gym/envs/classic_control/pendulum.py')
# env = gym.make("Pendulum-v0")
# args = arg_parser()
if seed:
    env.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
min_Val = torch.tensor(1e-7).float().to(device) # min value

directory = '/home/stoch-lab/Downloads/graphs' + env_name +'/'

In [4]:
class Replay_buffer():
    '''
    Code based on:
    https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py
    Expects tuples of (state, next_state, action, reward, done)
    '''
    def __init__(self, max_size= capacity):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def push(self, data):
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        x, y, u, r, d = [], [], [], [], []

        for i in ind:
            X, Y, U, R, D = self.storage[i]
            x.append(np.array(X, copy=False))
            y.append(np.array(Y, copy=False))
            u.append(np.array(U, copy=False))
            r.append(np.array(R, copy=False))
            d.append(np.array(D, copy=False))

        return np.array(x), np.array(y), np.array(u), np.array(r).reshape(-1, 1), np.array(d).reshape(-1, 1)


class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)

        self.max_action = max_action

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.max_action * torch.tanh(self.l3(x))
        return x


class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        self.l1 = nn.Linear(state_dim + action_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, 1)

        self.l4 = nn.Linear(state_dim + action_dim, 400)
        self.l5 = nn.Linear(400, 300)
        self.l6 = nn.Linear(300, 1)

    def forward(self, x, u):
        x1 = F.relu(self.l1(torch.cat([x, u], 1)))
        x1 = F.relu(self.l2(x1)) #Removing RELU from here
        x1 = self.l3(x1)

        x2 = F.relu(self.l4(torch.cat([x, u], 1)))
        x2 = F.relu(self.l5(x2))
        x2 = self.l6(x2)
        return x1, x2

    def Q1(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)
        return x1

class DDPG(object):
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-4)

        self.critic = Critic(state_dim, action_dim).to(device)
        # self.critic2 = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        # self.critic_target2 = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=1e-3)
        self.replay_buffer = Replay_buffer()
        self.writer = SummaryWriter(directory)

        self.num_critic_update_iteration = 0
        self.num_actor_update_iteration = 0
        self.num_training = 0

    def select_action(self, state, exploration_noise = 0.1):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        # print(state)
        action = self.actor(state).cpu().data.numpy().flatten()
        if exploration_noise != 0: 
            action = (action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0]))
        return action.clip(env.action_space.low, env.action_space.high)

    def update(self):

        for it in range(update_iteration):
            # Sample replay buffer
            x, y, u, r, d = self.replay_buffer.sample(batch_size)
            state = torch.FloatTensor(x).to(device)
            action = torch.FloatTensor(u).to(device)
            next_state = torch.FloatTensor(y).to(device)
            done = torch.FloatTensor(1-d).to(device)
            reward = torch.FloatTensor(r).to(device)

            noise = torch.FloatTensor(u).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-2.0, 2.0)
            
            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (done * gamma * target_Q).detach()

            # Get current Q estimates
            current_Q1, current_Q2 = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
            self.writer.add_scalar('Loss/critic_loss', critic_loss, global_step=self.num_critic_update_iteration)
            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()
            self.num_critic_update_iteration += 1

            if it % policy_freq == 0:
                # Compute actor loss
                actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
                
                # Optimize the actor 
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()

                # Update the frozen target models
                for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

                self.num_actor_update_iteration += 1

    def save(self):
        torch.save(self.actor.state_dict(), directory + 'actor.pth')
        torch.save(self.critic.state_dict(), directory + 'critic.pth')
        # print("====================================")
        # print("Model has been saved...")
        # print("====================================")

    def load(self):
        self.actor.load_state_dict(torch.load(directory + 'actor.pth'))
        self.critic.load_state_dict(torch.load(directory + 'critic.pth'))
        print("====================================")
        print("model has been loaded...")
        print("====================================")

In [ ]:
seeds = [30,61]
train_count = 0
train_thresh = 200 # Claim training is complete if max reward is achieved for 200 episodes in total
mode = 'train'
for i in range(len(seeds)):
    random_seed = seeds[i]
    env.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    agent = DDPG(state_dim, action_dim, max_action)
    ep_r = 0
#     if train_count > train_thresh:
#           print("Training finished")
#           break
    if (mode == 'train'):
#         if train_count > train_thresh:
#           print("Training finished")
#           break
        if load: agent.load()
        print("Seed Value:-", random_seed)
        for i in range(max_episode):
            energy_per_ep = []
            temp_energy = 0
#             if train_count > train_thresh:
#               print("Training finished")
#               break
            total_reward = 0
            step =0
            stepsInEachEp = 0
            rewards_per_step_list = []
            thetaValue = []
            thetaDotVal = []
            state = env.reset()         
            for t in range(1000):    ## episode length is fixed at 1000
                action = agent.select_action(state, exploration_noise = 0.2)
                next_state, reward, done, info = env.step(action)
                agent.replay_buffer.push((state, next_state, action, reward, np.float64(done)))
                temp_energy += abs(30*action[0]*(next_state[0] - state[0]))
                alpha = (next_state[3] - state[3]) * 50
                state = next_state
                env.render()
                step += 1
                # cbf = ((-2 * next_state[2] * next_state[3]) - ((next_state[2])**2))
                ReluCBF = -max(0.0, -((-2 * next_state[2] * next_state[3]) + ((0.01)**2 - (next_state[2])**2))) 
                # ReluCBF2 = -max(0.0, -((-2 * next_state[2] * next_state[3]) - 2 * next_state[3] * alpha + ((0.01)**2 - (next_state[2])**2 - (next_state[3])**2)) 
                # EnergyPenalty = - temp_energy
                total_reward += reward + ReluCBF
            
                rewards_per_step_list.append(reward) #Storing rewards at each step in an episode
                if ( done ):     # (int(total_reward) >= maxRewards) or
                    break
            energy_per_ep.append(temp_energy)
            print(total_reward,",", temp_energy)
            if total_reward > 980:
              train_count = train_count + 1
              #print(train_count)
#             if train_count > train_thresh:
#               print("Training finished")
#               break
            agent.update()
            if i % log_interval == 0:
                agent.save()
        env.close()

Seed Value:- 30
-123.91216501220723 , 0.36819614325250805
-171.1247917746776 , 40.45173080653802
-141.4913183440554 , 33.593263855979515
-163.4328162271679 , 69.03447662077832
-150.75425972638482 , 32.79319114771495
-117.4435642518511 , 17.054426573872863
-138.10522491296575 , 18.765748088078478
-173.48535619498784 , 25.8703352342232
-170.47226451785008 , 58.746730006331596
-137.51302482980427 , 19.431001223271334
-139.7982064962958 , 98.22099808773979
-163.90093185294455 , 51.38953171282872
-163.4726215117846 , 37.789110019222605
-140.01025193008343 , 4.876290219656943
-125.77125722425396 , 41.08647327067526
-127.96956476097336 , 79.17404418608183
-108.46929937700308 , 3.7958622467492704
-129.73176113319454 , 91.52056521909674
-139.10302030164647 , 5.461545257348423
-137.10634899160652 , 48.33352586667738
-130.42727489358793 , 1.9411996356649308
-134.8938246079107 , 56.100807530704444
-115.21017443874143 , 1.3659119646695652
-135.47202319087214 , 25.06029273017376
-137.60621909575485 

999.603917065914 , 34.03273408579004
-72.15687865607705 , 87.01010004461678
985.9618739123418 , 108.9324861735846
996.2733488372639 , 37.56768425383797
999.3315219624583 , 24.949783242621432
76.93734135736798 , 64.57000792755325
992.7725126833924 , 53.189031532324314
997.0103708385222 , 36.48496191891897
37.853803183451895 , 74.02705566795265
39.48106817986041 , 74.48878479274357
999.0211008887044 , 24.10089843797088
228.84518595279232 , 46.526586976190565
990.1196524985526 , 96.67824026275935
260.7958368532955 , 93.31526335067237
32.0307418593087 , 78.66639998796494
204.93070710471824 , 67.93713932071294
20.86624389119958 , 82.06247299075639
309.9119715972614 , 51.25628916041594
472.59210104905173 , 41.814057702894345
998.5696046852377 , 51.38692748943366
14.734264785211266 , 64.55901936878503
997.4595832280124 , 34.70666779707792
993.9185076673951 , 68.11068798746845
20.786585840230202 , 83.19538019928483
999.3563474731266 , 24.783799442607542
996.7620910512135 , 38.434464637198424
9

988.6489081418475 , 92.59602292274921
981.9212276666808 , 117.74353624282497
958.8132248109055 , 194.10532438756795
982.3140374553933 , 131.98929997472115
954.9264827686667 , 240.13419964862632
969.8221128276438 , 164.40583036231394
978.7233079150683 , 155.10698432552533
-18.20934282122158 , 121.39394252361693
984.9836664710278 , 119.5493115101493
930.8408103400394 , 368.94203280074345
964.1370160881619 , 191.40531932922676
960.3076403231661 , 210.4851187530068
941.0959736190196 , 268.17287828006715
918.6877182160281 , 364.4534936537269
854.326252231956 , 540.753718618104
933.257254442739 , 259.50177815905494
894.0797171146311 , 388.4966507540425
903.3145128505817 , 304.03848032832036
937.6102920816986 , 188.6039353048913
850.5613210740323 , 438.45966868153226
889.1584175706859 , 248.7355581201587
895.0829296373827 , 204.69832082649688
-43.227505255898635 , 120.53418618074124
785.4374135354195 , 478.87149534317876
768.9399040796743 , 589.8138883280462
680.7026861351017 , 960.3532163128

981.3360930210243 , 208.0898772000467
975.1956193992493 , 211.14648092023592
981.2960484334732 , 199.47581300705377
988.3337166092184 , 177.6463953044951
995.1512913570477 , 166.6645788074844
980.4745711211386 , 197.62915412706081
990.0580981560973 , 190.22738767754603
989.3917864631599 , 184.72162377340763
955.6599839531632 , 271.6936104547285
974.8913494695996 , 212.56687071906342
957.6134108957601 , 270.3517171694222
979.6515217699676 , 221.89942928053367
973.0826536435233 , 225.0363768164229
991.6152715716897 , 183.13348997380172
978.6407267382003 , 205.81693677480644
980.8243509385121 , 192.5218477579672
957.7288473279698 , 317.55967237816094
976.2474295270252 , 208.17133695713758
989.0445225507433 , 181.40171583079442
991.4844977614235 , 174.43823734904328
968.5169066779794 , 229.07300533607392
985.4639284700488 , 177.2566654031591
976.0152548737467 , 183.7024031012102
988.0851947049964 , 153.361552439265
980.8096868752425 , 173.63393017515662
975.0690353675637 , 215.754398820490

In [ ]:
"""
we can use x_threshold as reward penalty and increase the value of x_threshold to a large value.

"""

In [18]:
mode='test'
# thetaVal = [0, 0.174533, 0.349066, 0.523599, 0.698132, 0.872665, 0.959931, 1.0472, 1.13446, 1.22173, 1.309, 1.39626, 1.48353, 1.5708]
test_iteration = 1
if mode == 'test':
        agent.load
        for i in range(test_iteration):
            env.reset()
            state = env.state = env.unwrapped.state = np.array([0, 0, 1.0472, 0])
            total_pow = 0
            for t in range(1000):
                #print(state[3])
                action = agent.select_action(state)
                #np.abs(action*30*(next_state[0]-state[0]))
                #print(action)
                next_state, reward, done, info = env.step(np.float32(action))
                #ep_r +=  reward + (-2 * next_state[2] * next_state[3] - (next_state[2])**2)
                ReluCBF = -max(0.0, -((-2 * next_state[2] * next_state[3]) + ((0.01)**2 - (next_state[2])**2))) 
                ep_r += reward +ReluCBF
                env.render()
                temp_pow = abs(30*action[0]*(next_state[0] - state[0]))
                total_pow += temp_pow
                #x,v,theta,theta_dot,energy
                print(state[0], ",", state[1], ",", state[2],",", state[3],",", temp_pow)
                if done or t>=1000:
                    ep_r = 0
                    break
                state = next_state

0.0 , 0.0 , 1.0472 , 0.0 , 0.0
0.0 , 0.5490262603061468 , 1.0472 , -0.15715612004105325 , 0.28815259414430816
0.010980525206122936 , 1.028562949822771 , 1.0440568775991788 , -0.2621952830426255 , 0.6171377698936626
0.031551784202578356 , 1.5777267979022147 , 1.0388129719383263 , -0.4221581080626995 , 0.9466360787413287
0.06310632016062265 , 2.1271170424456787 , 1.0303698097770724 , -0.5868001345761953 , 1.173049456773997
0.10564866100953622 , 2.631940916437951 , 1.0186338070855485 , -0.7242974425349089 , 1.4883915505377654
0.15828747933829523 , 3.1502019748485823 , 1.0041478582348504 , -0.8817530368294603 , 1.6195294716186757
0.22129151883526688 , 3.621369754078754 , 0.9865127974982613 , -1.0130933162390356 , 2.1728218524472527
0.29371891391684196 , 4.172895598832762 , 0.9662509311734806 , -1.2241997736880115 , 2.2539586200397705
0.3771768258934972 , 4.669750073397992 , 0.9417669356997204 , -1.4059174243002859 , 2.801850044038796
0.4705718273614571 , 5.223366531257182 , 0.9136485872137

In [19]:
thetaDotDF #[0:20]

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.247207,0.648343,0.785136,0.818983,0.768646,0.759039,0.929901,0.892081,0.710674,0.839964,...,0.953006,-0.735614,0.927148,0.794952,0.847423,0.843483,0.843562,-0.059583,0.785003,0.905316
1,-0.242323,1.571500,1.545974,1.641058,1.225285,1.418540,1.761190,1.770694,1.484473,1.728212,...,0.312074,-0.232573,0.178888,1.259886,1.615440,1.017214,1.450185,0.361226,1.340437,1.771338
2,-0.174796,2.494871,2.387177,2.470834,1.997893,2.343147,2.685859,2.531400,2.174168,2.585361,...,0.375765,0.114883,1.048567,2.082048,1.035651,0.450614,2.070250,-0.270012,1.884379,1.109847
3,-0.156241,3.371696,3.240995,3.239006,2.741819,3.267038,3.608864,3.439441,2.819611,3.445889,...,-0.404282,-0.282014,0.334154,1.470142,1.829932,1.340161,1.399841,0.198826,1.765991,1.824812
4,-0.223814,4.292668,4.111867,4.102015,3.549088,4.187682,4.526490,4.088841,3.631700,4.113457,...,0.512156,-0.075045,0.856911,1.114496,0.973806,0.628990,1.006479,-0.247995,1.630822,1.046196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.066167,-0.184173,0.272973,0.274740,0.239187,0.319781,-0.461731,-0.257217,0.099431,-0.052162
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.036621,0.287138,-0.292787,0.069707,-0.244982,-0.244946,0.001097,0.265537,-0.068828,-0.085035
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.395150,-0.140446,0.377926,0.010938,0.340317,-0.272562,0.391994,-0.527314,0.110954,-0.267928
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.148005,0.081048,0.184011,0.145130,-0.471988,0.486879,-0.473747,0.182962,-0.042683,0.503175


In [5]:
def calc_time_step_mva():
    movingAvgOfStepRewards = []
    window_size = 1000
    sum_of_values = 0
    list_of_rewards_per_step_list_concat = np.concatenate(list_of_rewards_per_step_list, axis=0)
    for i in range(min(window_size, len(list_of_rewards_per_step_list_concat))):
        sum_of_values += list_of_rewards_per_step_list_concat[i]
    movingAvgOfStepRewards.append(sum_of_values/window_size)

    for i in range(window_size, len(list_of_rewards_per_step_list_concat)):
        sum_of_values += (list_of_rewards_per_step_list_concat[i] - list_of_rewards_per_step_list_concat[i-window_size])
        movingAvgOfStepRewards.append(sum_of_values/window_size)
    return movingAvgOfStepRewards
  

In [6]:
def calc_episodic_mva():
    movingAvgOfEpisodicRewards = []
    window_size = 5
    sum_of_values = 0

    for i in range(min(window_size, len(rewards_list))):
        sum_of_values += rewards_list[i]
    movingAvgOfEpisodicRewards.append(sum_of_values/window_size)

    for i in range(window_size, len(rewards_list)):
        sum_of_values += (rewards_list[i] - rewards_list[i-window_size])
        movingAvgOfEpisodicRewards.append(sum_of_values/window_size)
    return movingAvgOfEpisodicRewards

def relu(x):
	return max(0.0, x)

In [24]:
""" ctrl + shift + alt + r """

# import matplotlib.pyplot as plt
thetaDF = pd.DataFrame()
# list of name, degree, score
# nme = ["aparna", "pankaj", "sudhir", "Geeku"]
# deg = ["MBA", "BCA", "M.Tech", "MBA"]
# scr = [90, 40, 80, 98]
 
# # dictionary of lists
# dict = {'name': nme, 'degree': deg, 'score': scr}
     
# thetaDF = pd.DataFrame(dict)
listval = [i for i in range(100)]
listval2 = [i for i in range(80)]
listval3 = [i for i in range(90)]
# for i in ['listval', 'listval2']:
#     if(len())
#     thetaDF[i] = locals()[i]
thetaDF = pd.concat([thetaDF, pd.DataFrame(listval2)], axis = 1)
thetaDF = pd.concat([thetaDF, pd.DataFrame(listval)], axis = 1)
thetaDF = pd.concat([thetaDF, pd.DataFrame(listval3)], axis = 1)
thetaDF.set_axis([i for i in range(3)], axis=1, inplace = True)
thetaDF[0:81]
# abc = [1, 2, 3, 4]
# def1 = [5, 6, 8, 9]     # rename 'def' to 'def1' since 'def' is a Python keyword

# df = pd.DataFrame()

# for i in ['abc', 'def1']:
#    df[i] = locals()[i]  

# df

,0,1,2
0,0.0,0,0.0
1,1.0,1,1.0
2,2.0,2,2.0
3,3.0,3,3.0
4,4.0,4,4.0
...,...,...,...
76,76.0,76,76.0
77,77.0,77,77.0
78,78.0,78,78.0
79,79.0,79,79.0


In [8]:
# !mkdir /content/text_files
seeds = [30,42,58,61,63,78,93,101,39]
# seeds = [30]
fig_num = 0
maxRewards = 2500
mva_episodic_rewards = []
mva_step_rewards = []
for i in range(len(seeds)):
    random_seed = seeds[i]
    env.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    agent = DDPG(state_dim, action_dim, max_action)
    ep_r = 0
    if mode == 'test':
        agent.load
        for i in range(test_iteration):
            state = env.reset()
            for t in count():
                action = agent.select_action(state)
                next_state, reward, done, info = env.step(np.float32(action))
                ep_r += reward + (-2 * next_state[2] * next_state[3] - (next_state[2])**2)
                #               ep_r += reward
                env.render()
                if done or t >= max_length_of_trajectory:
                    print("Ep_i \t{}, the ep_r is \t{:0.2f}, the step is \t{}".format(i, ep_r, t))
                    ep_r = 0
                    break
                state = next_state
    elif mode == 'train':
        if load: agent.load()
        total_step = 0
        rewards_list = []
        steps_list = []
        thetaDF = pd.DataFrame()
        thetaDotDF = pd.DataFrame()
        stepsInEachEpisode = []
        list_of_rewards_per_step_list = [] # Storing a list of rewards per step in an episode for all episodes
        print("Seed Value:-", random_seed)
        for i in range(max_episode):
            total_reward = 0
            step =0
            stepsInEachEp = 0
            rewards_per_step_list = []
            thetaValue = []
            thetaDotVal = []
            state = env.reset()
#           theta = np.random.uniform(-np.pi/3, np.pi/3)
#           state[0] = np.cos(theta)
#           state[1] = np.sin(theta)          
            for t in range(1000):    ## episode length is fixed at 1000
                action = agent.select_action(state, exploration_noise = 0.2)
              # action = (action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0])).clip(
          #     env.action_space.low, env.action_space.high)
                next_state, reward, done, info = env.step(action)
#                 if render and i >= render_interval : 
                env.render()
                agent.replay_buffer.push((state, next_state, action, reward, np.float64(done)))

                state = next_state
                step += 1
#                 print("new reward value:-", ((-2 * next_state[2] * next_state[3]) - ((next_state[2])**2)))
#                 total_reward = total_reward + reward + ((-2 * next_state[2] * next_state[3]) - ((next_state[2])**2))
                Relu_x = max(0.0, -((-2 * next_state[2] * next_state[3]) + ((0.01)**2 - (next_state[2])**2)))
                total_reward = total_reward + reward + (-Relu_x)
                thetaValue.append(next_state[2])
                thetaDotVal.append(next_state[3])
#                 total_reward = total_reward + (-Relu_x)
#                 total_reward += reward
                rewards_per_step_list.append(reward) #Storing rewards at each step in an episode
                if ( done ):     # (int(total_reward) >= maxRewards) or
                    break
            total_step += step+1 #Adding +1 as env returns DONE = True 
            
            # store theta and thetaDot values in a Dataframe
            thetaDF = pd.concat([thetaDF, pd.DataFrame(thetaValue)], axis = 1)
            thetaDotDF = pd.concat([thetaDotDF, pd.DataFrame(thetaDotVal)], axis = 1)
#             thetaDF[i] = thetaValue
#             thetaDotDF[i] = thetaDotVal
            
            stepsInEachEp = step + 1
            rewards_list.append(total_reward) # Storing rewards for each episode
            steps_list.append(total_step) #No.of steps in that particular episode
            stepsInEachEpisode.append(stepsInEachEp)
            list_of_rewards_per_step_list.append(np.array(rewards_per_step_list))
            print("Total T:{} Episode: \t{} Total Reward: \t{:0.2f}".format(total_step, i, total_reward))
            agent.update()
                # "Total T: %d Episode Num: %d Episode T: %d Reward: %f
            if i % log_interval == 0:
                agent.save()
#         thetaDF.set_axis([i for i in range(1000)], axis=1, inplace = True)
#         thetaDF.to_csv('DDPG theta values with relu reward formulation pole angle 15 deg seed value ' +  str(int(random_seed)) + ' theta threshold is 90 degree')
#         thetaDotDF.set_axis([i for i in range(1000)], axis=1, inplace = True)
#         thetaDotDF.to_csv('DDPG theta_Dot values with relu reward formulation pole angle 15 deg seed value ' +  str(int(random_seed)) + ' theta threshold is 90 degree')
        with open('DDPG reward count with relu reward formulation pole angle 15 deg seed value ' +  str(int(random_seed)) + ' theta threshold is 90 degree', 'w') as f:
            for item in rewards_list:
                f.write("%s\n" % item)
                
        with open('DDPG steps count with relu reward formulation pole angle 15 deg seed value ' +  str(int(random_seed)) + ' theta threshold is 90 degree', 'w') as f:
            for item insteps_list:
                f.write("%s\n" % item)
        env.close()

Seed Value:- 30
Total T:37 Episode: 	0 Total Reward: 	-123.42
Total T:54 Episode: 	1 Total Reward: 	-168.75
Total T:71 Episode: 	2 Total Reward: 	-181.21
Total T:108 Episode: 	3 Total Reward: 	-118.44
Total T:125 Episode: 	4 Total Reward: 	-158.16
Total T:141 Episode: 	5 Total Reward: 	-141.38
Total T:156 Episode: 	6 Total Reward: 	-148.57
Total T:173 Episode: 	7 Total Reward: 	-178.63
Total T:190 Episode: 	8 Total Reward: 	-160.81
Total T:206 Episode: 	9 Total Reward: 	-165.20
Total T:223 Episode: 	10 Total Reward: 	-147.59
Total T:240 Episode: 	11 Total Reward: 	-180.11
Total T:256 Episode: 	12 Total Reward: 	-154.24
Total T:272 Episode: 	13 Total Reward: 	-160.13
Total T:289 Episode: 	14 Total Reward: 	-162.67
Total T:307 Episode: 	15 Total Reward: 	-141.16
Total T:325 Episode: 	16 Total Reward: 	-151.87
Total T:345 Episode: 	17 Total Reward: 	-152.28
Total T:367 Episode: 	18 Total Reward: 	-139.73
Total T:395 Episode: 	19 Total Reward: 	-140.01
Total T:426 Episode: 	20 Total Reward

Total T:17151 Episode: 	169 Total Reward: 	44.98
Total T:17398 Episode: 	170 Total Reward: 	68.14
Total T:17546 Episode: 	171 Total Reward: 	-18.61
Total T:17720 Episode: 	172 Total Reward: 	4.62
Total T:17883 Episode: 	173 Total Reward: 	-0.72
Total T:18057 Episode: 	174 Total Reward: 	-4.42
Total T:18234 Episode: 	175 Total Reward: 	95.74
Total T:18548 Episode: 	176 Total Reward: 	146.29
Total T:18752 Episode: 	177 Total Reward: 	164.32
Total T:19051 Episode: 	178 Total Reward: 	129.99
Total T:19258 Episode: 	179 Total Reward: 	41.35
Total T:19432 Episode: 	180 Total Reward: 	122.94
Total T:19708 Episode: 	181 Total Reward: 	204.33
Total T:19937 Episode: 	182 Total Reward: 	164.21
Total T:20121 Episode: 	183 Total Reward: 	-0.37
Total T:20519 Episode: 	184 Total Reward: 	343.53
Total T:20751 Episode: 	185 Total Reward: 	52.92
Total T:21060 Episode: 	186 Total Reward: 	131.97
Total T:21447 Episode: 	187 Total Reward: 	209.45
Total T:21841 Episode: 	188 Total Reward: 	232.28
Total T:22

Total T:138579 Episode: 	333 Total Reward: 	997.73
Total T:139580 Episode: 	334 Total Reward: 	997.49
Total T:139888 Episode: 	335 Total Reward: 	251.71
Total T:140889 Episode: 	336 Total Reward: 	998.67
Total T:141890 Episode: 	337 Total Reward: 	981.62
Total T:142891 Episode: 	338 Total Reward: 	995.45
Total T:143892 Episode: 	339 Total Reward: 	993.53
Total T:144893 Episode: 	340 Total Reward: 	996.74
Total T:145894 Episode: 	341 Total Reward: 	997.17
Total T:146895 Episode: 	342 Total Reward: 	990.96
Total T:147896 Episode: 	343 Total Reward: 	996.82
Total T:148897 Episode: 	344 Total Reward: 	994.90
Total T:149898 Episode: 	345 Total Reward: 	996.96
Total T:150899 Episode: 	346 Total Reward: 	996.09
Total T:151900 Episode: 	347 Total Reward: 	996.31
Total T:152901 Episode: 	348 Total Reward: 	972.09
Total T:153902 Episode: 	349 Total Reward: 	997.83
Total T:154903 Episode: 	350 Total Reward: 	995.59
Total T:155904 Episode: 	351 Total Reward: 	998.31
Total T:156905 Episode: 	352 To

Total T:297496 Episode: 	494 Total Reward: 	998.31
Total T:298497 Episode: 	495 Total Reward: 	999.32
Total T:299498 Episode: 	496 Total Reward: 	998.84
Total T:300499 Episode: 	497 Total Reward: 	999.03
Total T:301500 Episode: 	498 Total Reward: 	999.04
Total T:302501 Episode: 	499 Total Reward: 	998.88
Total T:303502 Episode: 	500 Total Reward: 	999.18
Total T:304503 Episode: 	501 Total Reward: 	998.95
Total T:305504 Episode: 	502 Total Reward: 	998.14
Total T:306505 Episode: 	503 Total Reward: 	998.04
Total T:307506 Episode: 	504 Total Reward: 	998.47
Total T:308507 Episode: 	505 Total Reward: 	998.89
Total T:308840 Episode: 	506 Total Reward: 	259.22
Total T:309841 Episode: 	507 Total Reward: 	996.28
Total T:310842 Episode: 	508 Total Reward: 	991.53
Total T:311843 Episode: 	509 Total Reward: 	996.30
Total T:312844 Episode: 	510 Total Reward: 	997.90
Total T:313845 Episode: 	511 Total Reward: 	997.26
Total T:314846 Episode: 	512 Total Reward: 	999.20
Total T:315847 Episode: 	513 To

Total T:449673 Episode: 	655 Total Reward: 	888.46
Total T:450214 Episode: 	656 Total Reward: 	539.37
Total T:451147 Episode: 	657 Total Reward: 	929.33
Total T:452148 Episode: 	658 Total Reward: 	996.69
Total T:453149 Episode: 	659 Total Reward: 	997.43
Total T:454150 Episode: 	660 Total Reward: 	998.18
Total T:455151 Episode: 	661 Total Reward: 	998.36
Total T:456152 Episode: 	662 Total Reward: 	998.42
Total T:457153 Episode: 	663 Total Reward: 	998.58
Total T:458154 Episode: 	664 Total Reward: 	998.02
Total T:459155 Episode: 	665 Total Reward: 	998.36
Total T:460156 Episode: 	666 Total Reward: 	998.33
Total T:461157 Episode: 	667 Total Reward: 	997.78
Total T:462158 Episode: 	668 Total Reward: 	998.40
Total T:463159 Episode: 	669 Total Reward: 	998.13
Total T:464160 Episode: 	670 Total Reward: 	997.11
Total T:465161 Episode: 	671 Total Reward: 	997.54
Total T:466162 Episode: 	672 Total Reward: 	994.32
Total T:467163 Episode: 	673 Total Reward: 	994.60
Total T:468164 Episode: 	674 To

...

Total T:147584 Episode: 	313 Total Reward: 	976.85
Total T:148585 Episode: 	314 Total Reward: 	960.07
Total T:149586 Episode: 	315 Total Reward: 	961.97
Total T:150587 Episode: 	316 Total Reward: 	962.39
Total T:151588 Episode: 	317 Total Reward: 	972.35
Total T:152589 Episode: 	318 Total Reward: 	962.05
Total T:153590 Episode: 	319 Total Reward: 	941.53
Total T:154591 Episode: 	320 Total Reward: 	920.30
Total T:155592 Episode: 	321 Total Reward: 	963.70
Total T:156593 Episode: 	322 Total Reward: 	940.61
Total T:157594 Episode: 	323 Total Reward: 	920.69
Total T:158595 Episode: 	324 Total Reward: 	934.08
Total T:159596 Episode: 	325 Total Reward: 	949.18
Total T:160597 Episode: 	326 Total Reward: 	908.47
Total T:161598 Episode: 	327 Total Reward: 	946.46
Total T:162599 Episode: 	328 Total Reward: 	946.65
Total T:163600 Episode: 	329 Total Reward: 	924.28
Total T:164601 Episode: 	330 Total Reward: 	916.39
Total T:165602 Episode: 	331 Total Reward: 	940.95
Total T:166603 Episode: 	332 To

Total T:301875 Episode: 	474 Total Reward: 	996.95
Total T:302876 Episode: 	475 Total Reward: 	995.80
Total T:303877 Episode: 	476 Total Reward: 	998.50
Total T:304878 Episode: 	477 Total Reward: 	998.08
Total T:305879 Episode: 	478 Total Reward: 	996.02
Total T:306880 Episode: 	479 Total Reward: 	996.68
Total T:307881 Episode: 	480 Total Reward: 	997.34
Total T:308882 Episode: 	481 Total Reward: 	998.95
Total T:309728 Episode: 	482 Total Reward: 	691.92
Total T:310729 Episode: 	483 Total Reward: 	999.79
Total T:311730 Episode: 	484 Total Reward: 	999.33
Total T:312731 Episode: 	485 Total Reward: 	999.56
Total T:313732 Episode: 	486 Total Reward: 	999.06
Total T:314733 Episode: 	487 Total Reward: 	999.12
Total T:315734 Episode: 	488 Total Reward: 	999.31
Total T:316735 Episode: 	489 Total Reward: 	999.64
Total T:317736 Episode: 	490 Total Reward: 	999.65
Total T:318737 Episode: 	491 Total Reward: 	999.58
Total T:319738 Episode: 	492 Total Reward: 	998.82
Total T:320739 Episode: 	493 To

Total T:450286 Episode: 	635 Total Reward: 	995.12
Total T:451287 Episode: 	636 Total Reward: 	996.45
Total T:452288 Episode: 	637 Total Reward: 	996.75
Total T:453289 Episode: 	638 Total Reward: 	995.39
Total T:454290 Episode: 	639 Total Reward: 	996.52
Total T:455291 Episode: 	640 Total Reward: 	999.26
Total T:456292 Episode: 	641 Total Reward: 	999.37
Total T:457293 Episode: 	642 Total Reward: 	996.06
Total T:458294 Episode: 	643 Total Reward: 	994.95
Total T:459295 Episode: 	644 Total Reward: 	999.14
Total T:460296 Episode: 	645 Total Reward: 	995.13
Total T:461297 Episode: 	646 Total Reward: 	999.00
Total T:462029 Episode: 	647 Total Reward: 	730.34
Total T:463030 Episode: 	648 Total Reward: 	993.43
Total T:464031 Episode: 	649 Total Reward: 	988.80
Total T:465032 Episode: 	650 Total Reward: 	979.89
Total T:466033 Episode: 	651 Total Reward: 	973.58
Total T:467034 Episode: 	652 Total Reward: 	981.62
Total T:468035 Episode: 	653 Total Reward: 	946.19
Total T:469036 Episode: 	654 To

Total T:590991 Episode: 	796 Total Reward: 	217.40
Total T:591204 Episode: 	797 Total Reward: 	49.85
Total T:591350 Episode: 	798 Total Reward: 	-10.29
Total T:591700 Episode: 	799 Total Reward: 	346.37
Total T:591923 Episode: 	800 Total Reward: 	218.52
Total T:592257 Episode: 	801 Total Reward: 	330.41
Total T:593258 Episode: 	802 Total Reward: 	995.48
Total T:594259 Episode: 	803 Total Reward: 	995.44
Total T:595260 Episode: 	804 Total Reward: 	961.43
Total T:596261 Episode: 	805 Total Reward: 	992.99
Total T:597262 Episode: 	806 Total Reward: 	992.26
Total T:598263 Episode: 	807 Total Reward: 	977.87
Total T:598384 Episode: 	808 Total Reward: 	-40.77
Total T:598510 Episode: 	809 Total Reward: 	-48.14
Total T:598720 Episode: 	810 Total Reward: 	203.72
Total T:598941 Episode: 	811 Total Reward: 	214.45
Total T:599230 Episode: 	812 Total Reward: 	281.85
Total T:599576 Episode: 	813 Total Reward: 	338.51
Total T:599822 Episode: 	814 Total Reward: 	239.66
Total T:600025 Episode: 	815 Tot

Total T:739025 Episode: 	957 Total Reward: 	995.95
Total T:740026 Episode: 	958 Total Reward: 	994.94
Total T:741027 Episode: 	959 Total Reward: 	997.43
Total T:742028 Episode: 	960 Total Reward: 	994.26
Total T:743029 Episode: 	961 Total Reward: 	995.83
Total T:744030 Episode: 	962 Total Reward: 	994.54
Total T:745031 Episode: 	963 Total Reward: 	995.90
Total T:746032 Episode: 	964 Total Reward: 	997.37
Total T:747033 Episode: 	965 Total Reward: 	996.13
Total T:748034 Episode: 	966 Total Reward: 	996.37
Total T:749035 Episode: 	967 Total Reward: 	994.19
Total T:750036 Episode: 	968 Total Reward: 	997.88
Total T:751037 Episode: 	969 Total Reward: 	996.38
Total T:752038 Episode: 	970 Total Reward: 	994.20
Total T:753039 Episode: 	971 Total Reward: 	993.49
Total T:754040 Episode: 	972 Total Reward: 	994.17
Total T:755041 Episode: 	973 Total Reward: 	995.12
Total T:756042 Episode: 	974 Total Reward: 	995.70
Total T:757043 Episode: 	975 Total Reward: 	997.36
Total T:758044 Episode: 	976 To

In [ ]:
# movingAvgOfEpisodicRewards = mva_episodic_rewards[0]
# plt.plot(np.arange(0, len(movingAvgOfEpisodicRewards), step=1), np.mean(mva_episodic_rewards, axis=0), color = 'black')

In [ ]:
# agent = DDPG(state_dim, action_dim, max_action)
# ep_r = 0
# if mode == 'test':
#     agent.load()
#     for i in range(test_iteration):
#         state = env.reset()
#         for t in count():
#             action = agent.select_action(state)
#             next_state, reward, done, info = env.step(np.float32(action))
#             ep_r += reward
#             env.render()
#             if done or t >= max_length_of_trajectory:
#                 print("Ep_i \t{}, the ep_r is \t{:0.2f}, the step is \t{}".format(i, ep_r, t))
#                 ep_r = 0
#                 break
#             state = next_state
# elif mode == 'train':
#     if load: agent.load()
#     total_step = 0
#     rewards_list = []
#     steps_list = []
#     list_of_rewards_per_step_list = [] # Storing a list of rewards per step in an episode for all episodes
#     for i in range(max_episode):
#         total_reward = 0
#         step =0
#         rewards_per_step_list = []
#         state = env.reset()
#         for t in count():
#             action = agent.select_action(state, exploration_noise = 0.2)
#             # action = (action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0])).clip(
#             #     env.action_space.low, env.action_space.high)

#             next_state, reward, done, info = env.step(action)
#             if render and i >= render_interval : env.render()
#             agent.replay_buffer.push((state, next_state, action, reward, np.float(done)))

#             state = next_state
#             if done:
#                 print(state)
#                   # print("Reached +100 !")
#                 break
#             step += 1
#             total_reward += reward
#             rewards_per_step_list.append(reward) #Storing rewards at each step in an episode
#         total_step += step+1 #Adding +1 as env returns DONE = True 
#         rewards_list.append(total_reward) # Storing rewards for each episode
#         steps_list.append(total_step) #No.of steps in that particular episode
#         list_of_rewards_per_step_list.append(np.array(rewards_per_step_list))
#         print("Total T:{} Episode: \t{} Total Reward: \t{:0.2f}".format(total_step, i, total_reward))
#         agent.update()
#         # "Total T: %d Episode Num: %d Episode T: %d Reward: %f

#         if i % log_interval == 0:
#             agent.save()


In [ ]:
# print(rewards_list)
# print(steps_list[-1])
# print(len(list_of_rewards_per_step_list))

In [ ]:
# #Rewards plotted across time-steps (Rewards at each step in the episodes v/s the steps)
# import matplotlib.pyplot as plt
# plt.plot(np.arange(start=0, stop=steps_list[-1] - len(steps_list), step=1), np.concatenate(list_of_rewards_per_step_list, axis=0))

In [ ]:
# #Total Rewards plotted across episodes (Total Reward in episode v/s the episodes)
# import matplotlib.pyplot as plt
# plt.plot(np.arange(0, len(steps_list), step=1), rewards_list)

In [ ]:
# #MVA Rewards plotted across time-steps (MVA of Rewards at each step in the episodes with window size = 100 v/s the steps)
# import matplotlib.pyplot as plt

# plt.plot(np.arange(start=0, stop=len(movingAvgOfStepRewards), step=1), movingAvgOfStepRewards)

In [ ]:
# #MVA Total Rewards plotted across episodes (Moving average over total episodic rewards with window-size = 5 v/s the episodes)
# import matplotlib.pyplot as plt
# plt.plot(np.arange(0, len(movingAvgOfEpisodicRewards), step=1), movingAvgOfEpisodicRewards)

In [ ]:
# print(rewards_list)

In [ ]:
# #Saving Rewards List
# with open('episodic_rewards_list_' + str(seed) +'_.txt', 'w') as file:
#     file.write('\n'.join(str(value) for value in rewards_list))

In [ ]:
# #Saving Step Wise Rewards List
# with open('step_rewards_list_' + str(seed)+ '_.txt', 'w') as file:
#     file.write('\n'.join(str(value) for value in list_of_rewards_per_step_list_concat))